In [2]:
import imagesize
import pandas as pd
import os

path = r'D:\0-Code\PG\2_sem\0_Dyplom\ai-capstone-proj\kaggle\input\CODEBRIM-balanced\train\defects'
files = os.listdir(path)

shape_dict = {
    'img': [],
    'width': [],
    'height': []
}
for file in files:
    if file.endswith('.png') or file.endswith('.jpg') or file.endswith('.jpeg'):
        shape = imagesize.get(os.path.join(path, file))
        shape_dict['img'].append(os.path.join(file))
        shape_dict['width'].append(shape[0])
        shape_dict['height'].append(shape[1])

df = pd.DataFrame(shape_dict)
df

,img,width,height
0,image_0000005_crop_0000001.png,331,1385
1,image_0000005_crop_0000001_copy00.png,331,1385
2,image_0000005_crop_0000001_copy01.png,331,1385
3,image_0000005_crop_0000001_copy02.png,331,1385
4,image_0000005_crop_0000002.png,397,2351
...,...,...,...
7019,image_0001599_crop_0000007.png,421,2709
7020,image_0001599_crop_0000008.png,1929,2359
7021,image_0001599_crop_0000008_copy00.png,1929,2359
7022,image_0001599_crop_0000008_copy01.png,1929,2359


In [3]:
df[df['height'] == df['height'].max()]

,img,width,height
145,image_0000199_crop_0000002.png,314,3999
150,image_0000201_crop_0000001.png,3223,3999
183,image_0000219_crop_0000001.png,1947,3999
184,image_0000219_crop_0000001_copy00.png,1947,3999
225,image_0000259_crop_0000001.png,1646,3999
491,image_0000368_crop_0000001.png,2747,3999
492,image_0000368_crop_0000001_copy00.png,2747,3999
493,image_0000368_crop_0000001_copy01.png,2747,3999
494,image_0000368_crop_0000001_copy02.png,2747,3999


In [ ]:
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import numpy as np

def pad_to_square(image: torch.Tensor, pad_value: float = 0.0) -> torch.Tensor:
    """
    Pad the smaller dimension of an image to make it square.

    Args:
        image (torch.Tensor): Input image tensor of shape (C, H, W) or (B, C, H, W)
        pad_value (float): Value to use for padding (default: 0.0)

    Returns:
        torch.Tensor: Padded square image with height = width
    """
    if len(image.shape) == 3:
        h, w, c = image.shape
        batch_mode = False
    elif len(image.shape) == 4:
        b, c, h, w = image.shape
        batch_mode = True
    else:
        raise ValueError(f"Expected 3D or 4D tensor, got shape {image.shape}")

    if h == w:
        return image

    diff = abs(h - w)
    pad_1 = diff // 2
    pad_2 = diff - pad_1

    # Determine which dimension to pad
    if h < w:
        padding = (0, 0, pad_1, pad_2)  # left, right, top, bottom
    else:
        padding = (pad_1, pad_2, 0, 0)  # left, right, top, bottom

    if batch_mode:
        return F.pad(image, padding, mode='constant', value=pad_value)
    else:
        return F.pad(image.unsqueeze(0), padding, mode='constant', value=pad_value).squeeze(0)

def pad_to_square_numpy(image: np.ndarray, pad_value: int = 0) -> np.ndarray:
    """
    Pad the smaller dimension of an image to make it square (NumPy version).

    Args:
        image (np.ndarray): Input image array of shape (H, W, C) or (H, W)
        pad_value (int): Value to use for padding (default: 0)

    Returns:
        np.ndarray: Padded square image with height = width
    """
    if len(image.shape) == 3:
        h, w, c = image.shape
    elif len(image.shape) == 2:
        h, w = image.shape
        c = None
    else:
        raise ValueError(f"Expected 2D or 3D array, got shape {image.shape}")

    if h == w:
        return image

    diff = abs(h - w)
    pad_1 = diff // 2
    pad_2 = diff - pad_1

    # Create padding configuration
    if h < w:
        if c is None:
            padding = ((pad_1, pad_2), (0, 0))
        else:
            padding = ((pad_1, pad_2), (0, 0), (0, 0))
    else:
        if c is None:
            padding = ((0, 0), (pad_1, pad_2))
        else:
            padding = ((0, 0), (pad_1, pad_2), (0, 0))

    return np.pad(image, padding, mode='constant', constant_values=pad_value)

def cv2_to_tensor(img,
                 normalize=True,
                 to_float=True,
                 channel_first=True,
                 device='cpu') -> torch.Tensor:
    """
    Convert a CV2 image to a PyTorch tensor.

    Args:
        img: CV2 image (numpy array) in BGR format
        normalize: If True, normalize pixel values to [0, 1]
        to_float: If True, convert to float32
        channel_first: If True, change to (C, H, W) format
        device: Target device for the tensor

    Returns:
        torch.Tensor: Processed image tensor
    """
    # Handle both RGB and grayscale images
    if len(img.shape) == 2:
        # Add channel dimension for grayscale images
        img = np.expand_dims(img, axis=-1)

    # Convert BGR to RGB
    if img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Convert to float and normalize if requested
    if to_float:
        img = img.astype(np.float32)
        if normalize:
            img = img / 255.0

    # Convert to tensor
    tensor = torch.from_numpy(img)

    # Rearrange dimensions from (H, W, C) to (C, H, W) if requested
    if channel_first:
        tensor = tensor.permute(2, 0, 1)

    return tensor.to(device)

# NumPy example
# Create a sample 200x100x3 array
import cv2
import matplotlib.pyplot as plt

torch_img = cv2.imread(os.path.join(path, 'image_0000199_crop_0000002.png'))
torch_img = cv2.cvtColor(torch_img, cv2.COLOR_BGR2RGB)

print(torch_img.shape)
torch_img = torch.from_numpy(torch_img)
print(torch_img.shape)
padded_torch_img = pad_to_square(torch_img)
print(padded_torch_img.shape)
plt.imshow(padded_torch_img)

# numpy_img = cv2.imread(os.path.join(path, 'image_0000199_crop_0000002.png'))
# padded_numpy = pad_to_square_numpy(numpy_img)
# print(f"Original numpy shape: {numpy_img.shape}")
# print(f"Padded numpy shape: {padded_numpy.shape}")
#
# plt.imshow(padded_numpy)

(3999, 314, 3)
torch.Size([3999, 314, 3])
torch.Size([3999, 314, 3688])


In [ ]:
os.makedirs('images', exist_ok=True)